# Importing all necessary libraries

In [ ]:
!pip install transformers
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=2f3b08e4530fc6e3eae0fb8f2cbdac86a09754d6b4c1bcf4ee725a0ed117a95c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import pandas as pd
from fastai.text.all import *
from transformers import *
#from blurr.data.all import *
#from blurr.modeling.all import *

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
from scipy import sparse
import numpy as np

from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
ERROR:root:Internal Python error in the inspect module.
Below is the traceback fr

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-230a290fb9dd>", line 3, in <cell line: 3>
    from transformers import *
  File "<frozen importlib._bootstrap>", line 1073, in _handle_fromlist
  File "<frozen importlib._bootstrap>", line 1075, in _handle_fromlist
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py", line 1137, in __getattr__
    value = getattr(module, name)
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py", line 1136, in __getattr__
    module = self._get_module(self._class_to_module[name])
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py", line 1146, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in impor

TypeError: ignored

In [ ]:
import torch
import numpy as np


from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Defining model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

model_name = 'facebook/bart-large-cnn'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder_max_length = 256
decoder_max_length = 64


#Pre-Processing

In [ ]:
import re
import string
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Mini Project/dataset.csv')

def remove_date(df):
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    df = df.applymap(lambda x: re.sub(pattern, '', str(x)))
    return df

urls = r'\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def remove_url(df):
  df = df.applymap(lambda x: re.sub(urls, '', str(x)))
  df = df.applymap(lambda x: re.sub('URL: ', '', str(x)))
  return df

foot = 'LIIofIndia: Copyright Policy | Disclaimers | Privacy Policy | Feedback'

def remove_foot(df):
  df = df.applymap(lambda x: re.sub(foot, '', str(x)))
  return df

def remove_punct(df):
    df = df.applymap(lambda x: x.translate(str.maketrans("", "", string.punctuation)))
    return df

# X = remove_date(X)
# y = remove_date(y)

# X = remove_url(X)
# y = remove_url(y)

# X = remove_foot(X)
# y = remove_foot(y)

data = remove_foot(data)
data = remove_date(data)
data = remove_url(data)
data = remove_punct(data)


In [ ]:
pd.options.display.max_colwidth = 500

In [ ]:
data

,Decisions,Headnote
0,HIDAYATULLAH MHIDAYATULLAH MDAYAL RAGHUBARCITATION 1964 AIR 1263 1964 SCR 6 172CITATOR INFO R 1977 SC2252 4ACTCriminal TrialPenal CodeConviction under s 304 Part IIIf can be read with s 34Intention KnowledgeIndian Penal Code 1860 45 of 1860 ss 34 35 38 and 304i Section 34 when it speaks of a criminal act done by several persons in furtherance of the common intention of all has regard not to the offence as a whole but to the criminal act that is to say the totality of the series of acts whi...,The six appellants were convicted under s 304 Part II with s 34 of the Indian Penal Code by the Sessions Judge add their appeal was summarily dismissed by the High Court On appeal by special leave it was contended that s 304 Part 11 could not be read with s 34 Indian Penal Code because the second part of s 304 excluded intention and was concerned with knowledge and the conviction was illegal
1,HIDAYATULLAH MHIDAYATULLAH MSHAH JCCITATION 1965 AIR 440 1964 SCR 6 181ACTRent ControlOrder for deposit of rent at interlocutory stageIf properDelhi Rent Control Act 1958 59 of 1958 s 151Indian Incometax Act 1922 11 of 1922 s465Ai that the order under subs 1 of s 15 is not a final order but is preliminary to the trial of the case and is made only wherer the rent has in fact not been paid For the purpose of an interim order it was not necessary that there should have been a fun trial and tha...,The respondent made an application against the appellant under s 14 of the Delhi Rent Control Act In reply the appellant pleaded 182 that the respondent had no right to recover rent from him as a notice under s 465A of the Indian Incometax Act had been issued by the Incometax Officer that the respondent had no locus standi as the property was in the custody of the Court and a receiver had been appointed and that his professional fees were agreed and be adjusted towards the rent dues The Rent...
2,AYYANGAR N RAJAGOPALA AYYANGAR N RAJAGOPALA SINHA BHUVNESHWAR PCJ SUBBARAO KDAYAL RAGHUBAR MUDHOLKAR JRCITATION 1964 AIR 1027 1964 SCR 6 213CITATOR INFO R 1965 SC 815 7 R 1970 SC 765 9 R 1971 SC 342 6 RF 1974 SC1185 16 R 1978 SC 840 46 E 1980 SC 303 1316 R 1983 SC 558 30 41 F 1989 SC1160 28 RF 1991 SC1557 31ACTRepresentation of the People Act 1951 43 of 1951 ss 80 81 3 903Election PetitionNo attestation in the copies of the petition that they are true copiesWhether there had been substantia...,The third respondent was declared elected to the Legislative Assembly of Andhra Pradesh in the general election held in 1962 The appellant a voter of the constituency filed an election petition challenging 214 the election of the third respondent on several grounds including corrupt practices The petition was accompanied by the requisite number of copies which were true copies and each of them bore the signature of the petitioner But there was no attestation at the foot of the copies that th...
3,AYYANGAR N RAJAGOPALA AYYANGAR N RAJAGOPALA SINHA BHUVNESHWAR PCJ SHAH JCCITATION 1965 AIR 295 1964 SCR 6 192CITATOR INFO R 1980 SC1655 5ACTCode of Civil Procedure Act Y of 1908 0 Ix r 9scope ofCrown Grants Act s 3Lease by GovernmentEffect of Applicability of s 41 of Transfer of Property Act IV of 1882 that the suit was substantially barred by 0 TX r9 The essential bundle of facts on which the plaintiffs based their title and their right to relief were identical in the two suits the propert...,The property covered by the Tea Estate was granted by the Government by way of lease in 1898 for 30 years In 1913 it was purchased by Azam Ali When he died in 1917 he left behind 8 sons 9 daughters and 3 widows The name of Ismail his eldest son was entered in the official records as next in succession Ismail borrowed considerable sums from National Agency Co Ltd and for securing the same deposited the title deeds of the Tea Estate on the footing that he was its full owner As the amount under...
4,AYYANGAR N RAJAGOPALA AYYANGAR N RAJAGOPALA SINHA BHUVNESHWAR PCJ SUBBARAO KDAYA

#Splitting and tokenizing

In [ ]:
X = list(data["Decisions"])
Y = list(data["Headnote"])

X_test = X[15:25]
Y_test = Y[15:25]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
Y_train_tokenized = tokenizer(Y_train, padding=True, truncation=True, max_length=512)
Y_val_tokenized = tokenizer(Y_val, padding=True, truncation=True, max_length=512)



print(X_train_tokenized)
# X_train_tokenized = X_train.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))
# Y_train_tokenized = Y_train.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))
# Y_val_tokenized = Y_val.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))
# X_val_tokenized = X_val.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))


# padding="max_length", max_length=encoder_max_length,
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets = None):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        input_ids = torch.tensor(self.inputs["input_ids"][index]).squeeze()
        target_ids = torch.tensor(self.targets["input_ids"][index]).squeeze()

        return {"input_ids": input_ids, "labels": target_ids}

train_dataset = Dataset(X_train_tokenized, Y_train_tokenized)
val_dataset = Dataset(X_val_tokenized, Y_val_tokenized)

{'input_ids': [[0, 16995, 2118, 6, 248, 3450, 725, 12027, 2747, 16995, 2118, 6, 248, 3450, 725, 12027, 2747, 25272, 387, 19142, 673, 6, 229, 4, 2547, 975, 9298, 2747, 6, 234, 4, 19545, 863, 3450, 5733, 2118, 2562, 2068, 6034, 35, 18202, 23464, 158, 4956, 18202, 4998, 500, 36, 134, 43, 29049, 347, 2068, 39261, 42962, 4832, 30455, 6200, 4998, 1558, 698, 36, 401, 43, 18044, 15002, 4998, 231, 3818, 36, 398, 43, 18044, 11724, 4998, 231, 4111, 36, 2481, 43, 13709, 35, 37697, 10465, 1783, 6, 504, 5479, 36, 26880, 290, 9, 504, 5479, 238, 579, 4, 28833, 1640, 398, 19281, 41761, 2273, 11, 2439, 6595, 1258, 9, 1637, 6, 5579, 8155, 16, 5579, 1121, 23861, 31, 3328, 9, 1637, 547, 7, 28, 29808, 5579, 42848, 15589, 2609, 25, 7, 145, 2273, 11, 2439, 6595, 1258, 6, 17331, 13, 5492, 4, 36, 118, 43, 20, 621, 54, 64, 28, 14914, 1720, 223, 579, 4, 25540, 1640, 398, 43, 16, 65, 6, 8155, 16, 11, 143, 169, 128, 3865, 37701, 108, 11, 5, 3210, 9, 5, 8637, 9, 2406, 88, 666, 50, 602, 66, 9, 5, 247, 3057, 19, 2098,

#Fine-Tuning model to the dataset

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    # learning_rate=3e-05,
    warmup_steps=5,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=5,
    save_total_limit=3
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
print(test_samples.inputs)

In [ ]:
model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": fals

#Generating summaries

In [ ]:
def generate_summary(X_test, model):
    inputs = tokenizer(
        X_test,
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

summaries_before_tuning = generate_summary(X_test, model_before_tuning)[1]
summaries_after_tuning = generate_summary(X_test, model)[1]

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.28.1"
}

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_leng

In [ ]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(Y_test)), headers=["Id", "Target summary"])
)
print("\nSource documents:\n")
print(tabulate(list(enumerate(X_test)), headers=["Id", "Document"]))

  Id  Summary after                                                                                                                                                                                                                                                                                                                                                                                                 Summary before
----  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Results

In [ ]:
!pip install rouge
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(summaries_after_tuning,list((Y_test)))



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
rouge2_score = []
rougeL_score = []

for i in range(0, 10):
    rouge2_score.append(scores[i]["rouge-2"]["f"])
    rougeL_score.append(scores[i]["rouge-l"]["f"])
print(np.mean(rouge2_score))
print(np.mean(rougeL_score))

0.0545486720210497
0.16981897655890146
